In [1]:
library(MatchIt)
library(dplyr)
library(tidyr)
library(sparklyr)
library(ggplot2)
library(ggridges)
library(gridExtra)
library(Hmisc)
require(tidyverse)
require(lmerTest)
require(grid)
library(effsize)
library(tableone)
library(labelled)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: lattice

Loading required package: survival

Loading required package: Formula


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ gridExtra::combine() masks dplyr::combine()
✖ dplyr::filter()      masks stats::filter()
✖ purrr::invoke()  

In [2]:
ENDPOINT = "cardiac"

In [3]:
config <- spark_config()
config$`sparklyr.shell.driver-memory` <- "8G"
config$`sparklyr.shell.executor-memory` <- "8G"
sc <- spark_connect(master = "local", config = config)
spark_tbl_handle <- spark_read_parquet(sc, "bolus", "/cluster/work/grlab/clinical/hirid2/research/faltysm/volume_challenge/patient_stay/2021-03-05")
df_stay <- collect(spark_tbl_handle)
spark_tbl_handle <- spark_read_parquet(sc, "bolus", "/cluster/work/grlab/clinical/hirid2/research/faltysm/volume_challenge/gap_closed/measurements_around_bolus/2020-12-05")
df_bolus <- collect(spark_tbl_handle)

if (ENDPOINT == "cardiac")
    spark_tbl_handle <- spark_read_parquet(sc, "bolus", "/cluster/work/grlab/clinical/hirid2/research/faltysm/volume_challenge/gap_closed/psm_cardiac/2020-12-30")
df_psm <- collect(spark_tbl_handle)

Warning message in file(input, encoding = "UTF-8"):
“'raw = FALSE' but '/cluster/home/config.yml' is not a regular file”
Warning message in readLines(con):
“cannot open file '/cluster/home/config.yml': Permission denied”


In [11]:
df_psm

patientid,bolusid,bolus,vm1_baseline,vm1_trend,vm1_0,vm1_300,vm1_600,vm1_900,vm1_1200,⋯,endindex,fluidamount,lenght,previous_fluid_1h,previous_fluid_2h,previous_fluid_30min,previous_fluid_total,startindex,vm1_criterion,vm5_criterion
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
100013,1,1,90.00000,0.0000000000,90.00000,90.00000,90.00000,90.00000,90.00000,⋯,36,300,1,303.87515,571.77216,169.60457,671.4150,36,1,1
100055,3,0,90.00000,0.0000000000,90.00000,90.00000,90.50000,90.00000,90.00000,⋯,91,265,6,219.97999,710.54158,73.94930,2366.3901,86,1,1
100075,1,0,89.00000,0.0000000000,89.00000,89.00000,89.00000,89.00000,89.00000,⋯,103,375,6,92.59736,185.19471,46.29868,1568.1771,98,1,1
100075,1,1,89.00000,0.0000000000,89.00000,89.00000,89.00000,89.00000,89.00000,⋯,56,375,6,149.04461,272.34636,87.39374,858.3444,51,1,1
100075,2,0,89.00000,0.0000000000,89.00000,89.00000,89.00000,89.00000,89.00000,⋯,92,375,6,92.59736,185.19471,46.29868,1483.2962,87,1,1
100084,2,0,99.43750,-0.0034313793,96.66667,93.33333,92.00000,90.66667,90.50000,⋯,198,270,3,0.00000,92.53925,0.00000,5844.3004,196,1,1
100084,2,1,101.53333,0.0050763566,98.50000,103.50000,103.00000,98.00000,94.66667,⋯,129,270,3,0.00000,749.58264,0.00000,4934.6653,127,1,1
100125,2,0,59.33333,0.0006019962,59.00000,60.66667,61.00000,60.00000,61.00000,⋯,220,600,1,0.00000,0.00000,0.00000,3864.8516,220,1,1
100125,2,1,89.00000,0.0000000000,89.00000,89.00000,89.00000,89.00000,89.00000,⋯,78,600,1,198.24189,863.62534,0.00000,1795.2932,78,1,1


In [4]:
df_patient <- df_psm %>%
        filter(ventilated_baseline > 0) %>%
        count(patientid)

In [5]:
patientid.ventilated <- function(pid) {
    results <- rep(FALSE, length(pid))
    df_patient <- df_bolus %>%
        filter(ventilated_baseline > 0) %>%
        count(patientid)
    
    for(i in 1:length(pid)) {
        if (nrow(df_patient[df_patient$patientid == pid[i],]) > 0)
            results[i] <- TRUE
        else
            results[i] <- FALSE
    }
    return(results)
}

In [6]:
#get overall group
df_overall <- df_stay %>%
    filter(invasive_hr_measurement_criterion & invasive_bp_measurement_criterion & age_criterion & study_drug_criterion & ecmo_criterion)
#filter for study group
if (ENDPOINT == "cardiac") {
    df_overall <- df_overall %>%
        filter(df_overall$adm_codeid %in% c(8699,8700,8701,9085,9086,9087,9088,9089))
}
df_overall$bolus = -1  #add a overall column encoded bolus -1
df_overall$ventilated_baseline = patientid.ventilated(df_overall$patientid)

#get psm bolus and population group
dt <- df_stay %>%
    merge(df_psm,by="patientid") %>%
    select(c('weight', 'outcome_death', 'sex','age_at_admission','apacheII', 'sepsis', 'height', 'emergency', 'adm_diag', 'lenght_of_stay', 'bolus', 'ventilated_baseline', 'patientid', 'adm_codeid'))
dt <- dt %>%
    rbind(df_overall %>% select(c('weight', 'outcome_death', 'sex','age_at_admission','apacheII', 'sepsis', 'height', 'emergency', 'adm_diag', 'lenght_of_stay', 'bolus', 'ventilated_baseline', 'patientid', 'adm_codeid')))

dt$bolus = as.factor(dt$bolus)
dt$sex = as.factor(dt$sex)
dt$ventilated_baseline = as.factor(dt$ventilated_baseline)
dt$no_emergency = as.factor(!dt$emergency)
dt$no_sepsis = as.factor(!dt$sepsis)
dt$outcome_death = as.logical(dt$outcome_death)
dt$adm_diag_group <- cut(dt$adm_diag, 
                       breaks = c(-Inf, 10, 16, Inf), 
                       labels = c("Medical", "Surgical", "Unknown"),
                       right = FALSE)
dt$adm_diag = as.factor(dt$adm_diag)
dt$lenght_of_stay = dt$lenght_of_stay/3600/24

#colum selection
columns_sel = c('sex','age_at_admission', 'height', 'weight', 'apacheII', 'no_sepsis', 'no_emergency', 'adm_diag', 'adm_codeid', 'lenght_of_stay', 'bolus', 'ventilated_baseline', 'outcome_death')
if (ENDPOINT == "cardiacs") {
    columns_Sel = c('sex','age_at_admission', 'height', 'weight', 'apacheII', 'no_sepsis', 'no_emergency', 'adm_codeid', 'lenght_of_stay', 'bolus', 'ventilated_baseline', 'outcome_death')
}

dat <- dt %>%
  select(c('weight', 'outcome_death', 'sex','age_at_admission','apacheII', 'no_sepsis', 'height', 'no_emergency', 'adm_diag', 'adm_codeid', 'lenght_of_stay', 'bolus', 'ventilated_baseline')) %>%
  mutate(sex = recode_factor(sex, 
                           `M` = "Males", 
                           `F` = "Females",
                           `U` = "Unknown"),
         adm_diag = recode_factor(adm_diag, 
                         `4` = "Neurological medical", 
                         `11` = "Cardiovascular surgical", 
                         `14` = "Neurological surgical", 
                         `1` = "Cardiovascular medical",
                         `2` = "Respiratory medical",
                         `3` = "Gastroenterological medical",
                         `6` = "Others",
                         `7` = "Others",
                         `8` = "Others",
                         `9` = "Others",
                         `12` = "Others",
                         `13` = "Others",
                         `15` = "Others",
                         `16` = "Others",
                         `17` = "Others",
                         `18` = "Others",
                         `5` = "Others"),
         adm_codeid = recode_factor(adm_codeid, 
                         `8699` = "CABG", 
                         `8700` = "Heart valve surgery", 
                         `8701` = "CABG + Heart valve surgery", 
                         `9085` = "CABG",
                         `9086` = "CABG",
                         `9087` = "Heart valve surgery",
                         `9088` = "CABG + Heart valve surgery",
                         `9089` = "CABG + Heart valve surgery"),
         no_emergency = recode_factor(no_emergency,    
                         'TRUE' = "Elective",
                         'FALSE' = "Emergency"),
         bolus = recode_factor(bolus, 
                         '-1' = "Overall",
                         '0' = "Control", 
                         '1' = "Bolus"),
         ventilated_baseline = recode_factor(ventilated_baseline, 
                         '0' = "No",
                         '1' = "Yes", 
                         '2' = "Yes"),
         no_sepsis = recode_factor(no_sepsis, 
                         'TRUE' = "No", 
                         'FALSE' = "Yes")) %>% 
  select(all_of(columns_sel))

In [7]:
var_label(dat) = list(
  sex = "Gender",
  age_at_admission = "Age at admission (years)",
  apacheII = "APACHE II score",
  no_sepsis = "Septic Shock",
  no_emergency = "Admission Type Emergency",
  adm_diag = "APACHE diagnostic group",
  adm_codeid = "Cardiac surgery types",
  ventilated_baseline = "Mechanical ventilation",
  lenght_of_stay = "ICU Length of stay (days)",
  weight = "Weight (kg)",
  height = "Height (cm)",
  outcome_death = "ICU mortality")

In [8]:
dat %>% CreateTableOne(
    vars = select(dat, -bolus) %>% names(), 
    test = FALSE,
    data = ., strata="bolus", addOverall=FALSE) -> table_one
print(table_one, varLabels = TRUE)

                                                       Stratified by bolus
                                                        Overall        
  n                                                       8440         
  Gender = Females (%)                                    2176 ( 25.8) 
  Age at admission (years) (mean (SD))                   66.93 (11.45) 
  Height (cm) (mean (SD))                               170.89 (20.18) 
  Weight (kg) (mean (SD))                                80.35 (17.67) 
  APACHE II score (mean (SD))                            17.39 (6.23)  
  Septic Shock = Yes (%)                                    12 (  0.1) 
  Admission Type Emergency = Emergency (%)                 486 (  5.8) 
  APACHE diagnostic group = Cardiovascular surgical (%)   8440 (100.0) 
  Cardiac surgery types (%)                                            
     CABG                                                 3723 ( 44.1) 
     Heart valve surgery                                  308

In [9]:
if (ENDPOINT == "cardiac") {
    output_file <- "./results/cardiac/table1.csv"
    print ("Processing cardiac psm")
}
a <- print(table_one, varLabels = TRUE, smd = FALSE, test = FALSE, dropEqual = TRUE, noSpaces = TRUE, printToggle = FALSE)
write.csv(a, file = output_file)

[1] "Processing cardiac psm"


In [11]:
#paced proportions (patient HR 90+-2/min in outcome region)
tolerance = 2
column = 'vm1'
df_psm$paced=(apply(df_psm[,c(paste(column, "baseline", sep="_"), paste(column, "0", sep="_"), paste(column, "300", sep="_"), paste(column, "600", sep="_"), paste(column, "900", sep="_"), paste(column, "1200", sep="_"), paste(column, "1500", sep="_"), paste(column, "1800", sep="_"), paste(column, "2100", sep="_"), paste(column, "2400", sep="_"), paste(column, "2700", sep="_"), paste(column, "3000", sep="_"), paste(column, "3300", sep="_"))], 1, median) - 90)<=tolerance
prop.table(table(df_psm$paced,df_psm$bolus,dnn=list("Paced","Bolus")))

sum(df_psm$paced)/sum(!is.na(df_psm$paced))

       Bolus
Paced           0         1
  FALSE 0.1504759 0.1380179
  TRUE  0.3495241 0.3619821

[1] 0.7115062